In [1]:
import os
import json
import numpy as np
import robotic as ry
from tqdm import tqdm
from lookup import features, contraint_types
from komo_generators import generate_random_komo, generate_random_pick, generate_random_pivot
from random_kitchen_generator import generate_random_kitchen, load_single_from_type

In [2]:
C = generate_random_kitchen(seed=89032, root_path="/home/denis/rai-robotModels/robocasa/")
frame_names = C.getFrameNames()
features["qItself"][1] = len(C.getJointState())
object_frames = []
with open("random_objs.g", 'r', encoding='utf-8') as file:
    for line in file:
        words = line.strip().split()
        if words:  # Ensure the line is not empty
            object_frames.append(words[0])
C = load_single_from_type(C, pos=[.5, 1., 1.2], root_path=f"/home/denis/rai-robotModels/robocasa/rai_jointed/fixtures/sinks")

import time
np.random.seed(int(time.time()))

Seed: 89032
Files found inside /home/denis/rai-robotModels/robocasa/rai_jointed/fixtures/sinks: 10
/home/denis/rai-robotModels/robocasa/rai_jointed/fixtures/sinks/white_sink/joint_scene.g


## Generate KOMO texts

In [3]:
# Special Features (for pure random KOMOs)
lucky_frames = ["r_gripper", "r_palm"]
lucky_features = ["positionDiff", "position", "vectorX", "vectorY", "vectorZ", "quaternion", "negDistance"]

# Special Features (for random manipulation KOMOs)
end_effector_frames = ["r_gripper", "l_gripper"]
mobile_base_frames = ["omnibase"]
# object_frames = ["kebabs_8", "can_6", "pizza_cutter_2", "bell_pepper_0", "kebabs_13"]  # Frames that are not attached to anything that can be manipulated with basic pick, place, push, etc.
joint_frames = ["spout_joint", "handle_joint"]
jointed_frames = ["spout_visual0", "handle_visual0"]  # Frames that have a parent / grandparent / great-grandparent / etc. that is a joint (Not including robot)
world_attach_frames = ["floor"]  # Like table in manipulation.py, some sort of world frame that tells us the world rotation and for us to place stuff after a mode switch.

In [4]:
# komo_generator = lambda: generate_random_komo(frame_names, features, lucky_frames, lucky_features, prob_mode_switch=0)
komo_generator = lambda: generate_random_pick(end_effector_frames,
                                              object_frames,
                                              world_attach_frames,
                                              frame_names,
                                              drop_prob=0,
                                              specific_orientation_prob=0,
                                              hard_grasp_prob=.1,
                                              specific_place_prob=0,
                                              place_prob=1,
                                              endeff_pass_prob=1,
                                              repeat_pass_prob=1,
                                              accumulated_collisions=False)
# komo_generator = lambda: generate_random_pivot(end_effector_frames, jointed_frames, joint_frames, slices=8)
# def komo_generator():
#     if np.random.random() < .5:
#         text = generate_random_pivot(end_effector_frames, jointed_frames, joint_frames, slices=8)
#     else:
#         text = generate_random_pick(end_effector_frames,
#                                     object_frames,
#                                     world_attach_frames,
#                                     frame_names,
#                                     drop_prob=0,
#                                     specific_orientation_prob=.3,
#                                     specific_place_prob=.1,
#                                     place_prob=1)
#     return text

In [5]:
for i in range(10):
    komo_text = komo_generator()
    print(komo_text)

# Pick object bagel_8 with endeffector r_gripper and place it on top of floor.
# Follow the endeff pass sequence: r_gripper, l_gripper, r_gripper, l_gripper, r_gripper, l_gripper, r_gripper, l_gripper, r_gripper.
komo = ry.KOMO(C, 11, 1, 2, False)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo.addObjective([], ry.FS.position, ['omnibase'], ry.OT.sos, [1e0], [], 1)
komo.addObjective([1], ry.FS.negDistance, ['r_gripper', 'bagel_8'], ry.OT.eq, [1e1], [0.001])
komo.addModeSwitch([1, 2], ry.SY.stable, ['r_gripper', 'bagel_8'], True)
komo.addObjective([2], ry.FS.negDistance, ['l_gripper', 'bagel_8'], ry.OT.eq, [1e1], [0.031])
komo.addModeSwitch([2, 3], ry.SY.stable, ['l_gripper', 'bagel_8'], False)
komo.addObjective([3], ry.FS.negDistance, ['r_gripper', 'bagel_8'], ry.OT.eq, [1e1], [0.025])
komo.addModeSwitch([3, 4], ry.SY.stable, ['r_gripper', 'bagel_8'], False)
ko

In [6]:
komo_samples = 1000

outfolder = "./komo_samples"
text_outfolder = os.path.join(outfolder, "texts")
if not os.path.exists(text_outfolder):
    os.makedirs(text_outfolder)
else:
    for file in os.listdir(text_outfolder):
        file_path = os.path.join(text_outfolder, file)
        os.remove(file_path)

for i in tqdm(range(komo_samples)):
    komo_text = komo_generator()
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "w") as file:
        file.write(komo_text)

100%|██████████| 1000/1000 [00:00<00:00, 9482.40it/s]


## Classify KOMO texts

In [7]:
total_komos = 0
total_feasible_komos = 0
total_error_komos = 0

feasible_komos = []
error_komos = []

for i in tqdm(range(komo_samples)):
    
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()

    # print(f"Processing file {filename}...")
    total_komos += 1
    
    # try:
    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
    # except:
    #     print(komo_text)
    #     exit()
    #     total_error_komos += 1
    #     error_komos.append(i)
    #     continue
    
    if ret.feasible:
        total_feasible_komos += 1
        feasible_komos.append(i)
    
    if (i+1)%(komo_samples//10) == 0:
        print(f"Total feasible komos: {total_feasible_komos}")
        print(f"Total error komos: {total_error_komos}")
        print(f"Total komos: {total_komos}")
        print(f"{(total_feasible_komos/total_komos)*100:.2f}% feasible")

  1%|          | 11/1000 [00:05<07:41,  2.14it/s]

-- pairCollision.cpp:libccd:365(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:370(0) WARNING: but GJK says intersection


  7%|▋         | 71/1000 [00:41<07:09,  2.16it/s]

-- pairCollision.cpp:libccd:365(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:370(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:365(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:370(0) WARNING: but GJK says intersection


 10%|█         | 100/1000 [00:59<06:38,  2.26it/s]

...

-- pairCollision.cpp:libccd:365(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:370(0) WARNING: but GJK says intersection


 90%|█████████ | 900/1000 [08:11<01:08,  1.46it/s]

Total feasible komos: 1
Total error komos: 0
Total komos: 900
0.11% feasible


100%|██████████| 1000/1000 [09:03<00:00,  1.84it/s]

Total feasible komos: 1
Total error komos: 0
Total komos: 1000
0.10% feasible


In [8]:
result = f"""Total feasible komos - Total error komos - Total komos: 
{total_feasible_komos} - {total_error_komos} - {total_komos}. 
({(total_feasible_komos/total_komos)*100:.2f}% feasible)"""
print(result)
print(feasible_komos)
results_file_path = os.path.join(outfolder, "feasibles.json")
results_dict = {
    "feasible_komos": feasible_komos,
    "error_komos": error_komos,
}
json.dump(results_dict, open(results_file_path, 'w'))

Total feasible komos - Total error komos - Total komos: 
1 - 0 - 1000. 
(0.10% feasible)
[744]


## Generate slices

In [ ]:
for i in tqdm(range(komo_samples)):
    
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()

    # print(f"Processing file {filename}...")
    total_komos += 1
    
    # try:
    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
    # except:
    #     print(komo_text)
    #     exit()
    #     total_error_komos += 1
    #     error_komos.append(i)
    #     continue
    
    if ret.feasible:
        total_feasible_komos += 1
        feasible_komos.append(i)
    
    if (i+1)%(komo_samples//10) == 0:
        print(f"Total feasible komos: {total_feasible_komos}")
        print(f"Total error komos: {total_error_komos}")
        print(f"Total komos: {total_komos}")
        print(f"{(total_feasible_komos/total_komos)*100:.2f}% feasible")

## Visualize feasible KOMOs

In [9]:
# outfolder = "komo_samples_pick"
# text_outfolder = os.path.join(outfolder, "text")
# feasible_komos = json.load(open(os.path.join(outfolder, "feasibles.json"), 'r'))["feasible_komos"]

feasibe_samples_count = 10
if len(feasible_komos) < feasibe_samples_count:
    feasibe_samples = feasible_komos.copy()
else:
    feasibe_samples = np.random.choice(feasible_komos, feasibe_samples_count, replace=False)
    print(feasibe_samples)
    feasibe_samples = [6054, 3687, 7429, 6099, 95, 1401, 305, 424, 8623, 5929, 5736]

for i in feasibe_samples:

    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()
    print(f"__________ SAMPLE {i} __________")
    print(komo_text)

    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()

    komo.view(True)
    komo.view_play(True, delay=.5)

__________ SAMPLE 744 __________
# Pick object alcohol_5 with endeffector l_gripper and place it on top of cameraWrist.
# Follow the endeff pass sequence: l_gripper, r_gripper.
komo = ry.KOMO(C, 3, 1, 2, False)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo.addObjective([], ry.FS.position, ['omnibase'], ry.OT.sos, [1e0], [], 1)
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'alcohol_5'], ry.OT.eq, [1e1])
komo.addModeSwitch([1, 2], ry.SY.stable, ['l_gripper', 'alcohol_5'], True)
komo.addObjective([2], ry.FS.positionDiff, ['r_gripper', 'alcohol_5'], ry.OT.eq, [1e1])
komo.addModeSwitch([2, 3], ry.SY.stable, ['r_gripper', 'alcohol_5'], False)
komo.addObjective([3], ry.FS.positionRel, ['cameraWrist', 'alcohol_5'], ry.OT.ineq, [0, 0, -1e1])
komo.addObjective([3], ry.FS.negDistance, ['cameraWrist', 'alcohol_5'], ry.OT.ineq, [-1e1], [-0.04])
komo.addObjectiv